# 🎨 MCA Villager Skin Generator (Clean Version)

MCA Rebornの村人と同様のスキンを**インタラクティブに**生成するツールです。

## 📋 使い方
1. **インポートセル**を実行
2. **パラメータ設定**で好みの値を調整
3. **「🎨 スキン生成」**ボタンをクリック
4. **「🖼️ 画像生成」**ボタンをクリック
5. 結果を確認

## 🎛️ 調整可能なパラメータ
- **性別**: 男性・女性・ランダム
- **特性**: ヘテロクロミア（オッドアイ）・アルビノ
- **肌の色**: ヘモグロビン・メラニン値
- **髪の色**: ユーメラニン・フェオメラニン・RGBカスタムカラー

## 1. ライブラリのインポート

In [ ]:
from PIL import Image
from IPython.display import display
from classes.skin_info import SkinInfo
from classes.generate_villager_skin import GenerateVillagerSkin
import ipywidgets as widgets
from IPython.display import clear_output
import numpy as np

print("✅ ライブラリのインポート完了！")

## 2. パラメータ設定とスキン生成

In [ ]:
# 性別選択
gender_dropdown = widgets.Dropdown(
    options=[('ランダム', 0), ('男性', 1), ('女性', 2)],
    value=0,
    description='性別:',
    style={'description_width': 'initial'}
)

# 特性設定
hetero_checkbox = widgets.Checkbox(
    value=False,
    description='ヘテロクロミア（オッドアイ）'
)

albinism_checkbox = widgets.Checkbox(
    value=False,
    description='アルビノ'
)

# 肌の色設定
hemoglobin_slider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.01,
    description='ヘモグロビン:',
    style={'description_width': 'initial'}
)

melanin_slider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.01,
    description='メラニン:',
    style={'description_width': 'initial'}
)

# 髪の色設定
eumelanin_slider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.01,
    description='ユーメラニン:',
    style={'description_width': 'initial'}
)

pheomelanin_slider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.01,
    description='フェオメラニン:',
    style={'description_width': 'initial'}
)

# RGBカスタムカラー
hair_red_slider = widgets.IntSlider(
    value=128,
    min=0,
    max=255,
    description='髪の色(R):',
    style={'description_width': 'initial'}
)

hair_green_slider = widgets.IntSlider(
    value=128,
    min=0,
    max=255,
    description='髪の色(G):',
    style={'description_width': 'initial'}
)

hair_blue_slider = widgets.IntSlider(
    value=128,
    min=0,
    max=255,
    description='髪の色(B):',
    style={'description_width': 'initial'}
)

# 出力エリア
output_area = widgets.Output()
image_output_area = widgets.Output()

print("✅ ウィジェットの準備完了！")

In [ ]:
# ランダム生成関数
def randomize_params(button):
    gender_dropdown.value = 0  # ランダム
    hetero_checkbox.value = False
    albinism_checkbox.value = False
    hemoglobin_slider.value = np.random.uniform(0.0, 1.0)
    melanin_slider.value = np.random.uniform(0.0, 1.0)
    eumelanin_slider.value = np.random.uniform(0.0, 1.0)
    pheomelanin_slider.value = np.random.uniform(0.0, 1.0)
    hair_red_slider.value = np.random.randint(0, 256)
    hair_green_slider.value = np.random.randint(0, 256)
    hair_blue_slider.value = np.random.randint(0, 256)

# スキン情報生成関数
def create_skin_info():
    params = {}
    
    # 性別（ランダムでない場合のみ設定）
    if gender_dropdown.value != 0:  # 0はランダム
        params["gender"] = gender_dropdown.value
    
    # 特性
    params["hetero"] = hetero_checkbox.value
    params["albinism"] = albinism_checkbox.value
    
    # 肌の色パラメータ
    params["hemoglobin"] = hemoglobin_slider.value
    params["melanin"] = melanin_slider.value
    
    # 髪の色パラメータ
    params["eumelanin"] = eumelanin_slider.value
    params["pheomelanin"] = pheomelanin_slider.value
    params["hair_color_blue"] = hair_blue_slider.value
    params["hair_color_green"] = hair_green_slider.value
    params["hair_color_red"] = hair_red_slider.value
    
    return SkinInfo(**params)

# スキン情報生成ボタンの処理
def generate_skin_info_handler(button):
    with output_area:
        clear_output(wait=True)
        print("🎨 スキン情報生成中...")
        
        try:
            global skin_info
            skin_info = create_skin_info()
            skin_info.generate_skin_info()
            
            print("✅ スキン情報生成完了！")
            print(f"📋 生成されたスキン情報:")
            print(skin_info)
            
        except Exception as e:
            print(f"❌ エラーが発生しました: {e}")

# 画像生成ボタンの処理
def generate_image_handler(button):
    with image_output_area:
        clear_output(wait=True)
        print("🖼️ スキン画像生成中...")
        
        try:
            if 'skin_info' not in globals():
                print("❌ 先に「🎨 スキン生成」ボタンを押してください。")
                return
                
            global villager_skin
            skin_generator = GenerateVillagerSkin(skin_info)
            villager_skin = skin_generator.generate()
            
            print("✅ スキン画像生成完了！")
            print("🎨 生成されたスキン画像:")
            display(villager_skin)
            
            print("🔍 4倍サイズで詳細表示:")
            size = (villager_skin.size[0] * 4, villager_skin.size[1] * 4)
            display(villager_skin.resize(size, Image.NEAREST))
            
            print("💾 画像を保存する場合:")
            print("villager_skin.save('custom_villager_skin.png')")
            
        except Exception as e:
            print(f"❌ エラーが発生しました: {e}")

print("✅ 関数の準備完了！")

In [ ]:
# ボタンの定義
random_button = widgets.Button(
    description='🎲 ランダム生成',
    button_style='info',
    tooltip='全パラメータをランダムに設定'
)

skin_info_button = widgets.Button(
    description='🎨 スキン生成',
    button_style='success',
    tooltip='設定したパラメータでスキン情報を生成'
)

image_button = widgets.Button(
    description='🖼️ 画像生成',
    button_style='warning',
    tooltip='スキン情報から画像を生成'
)

# イベントハンドラの設定
random_button.on_click(randomize_params)
skin_info_button.on_click(generate_skin_info_handler)
image_button.on_click(generate_image_handler)

print("✅ ボタンの準備完了！")

## 3. パラメータ設定と操作

In [ ]:
# ウィジェットの表示
print("🎛️ パラメータ設定:")
display(widgets.VBox([
    gender_dropdown,
    widgets.HTML("<b>特性:</b>"),
    widgets.HBox([hetero_checkbox, albinism_checkbox]),
    widgets.HTML("<b>肌の色:</b>"),
    hemoglobin_slider,
    melanin_slider,
    widgets.HTML("<b>髪の色:</b>"),
    eumelanin_slider,
    pheomelanin_slider,
    hair_red_slider,
    hair_green_slider,
    hair_blue_slider,
    widgets.HTML("<br>"),
    widgets.HBox([random_button, skin_info_button, image_button])
]))

print("📋 スキン情報生成結果:")
display(output_area)

print("🖼️ 画像生成結果:")
display(image_output_area)

## 4. 詳細情報表示

In [ ]:
# スキンの詳細情報表示
def show_detailed_info():
    try:
        if 'skin_info' in globals() and skin_info is not None:
            print("📋 スキンの詳細パラメータ情報:")
            display(skin_info.to_dataframe())
            
            print("\n🎯 主なパラメータの意味:")
            print("• gender: 性別 (1=男性, 2=女性)")
            print("• hetero: ヘテロクロミア（オッドアイ）")
            print("• albinism: アルビノ")
            print("• hemoglobin/melanin: 肌の色を決定する要素")
            print("• eumelanin/pheomelanin: 髪の色を決定する要素")
            print("• hair_color_*: 髪のRGBカスタムカラー")
        else:
            print("❌ skin_infoが見つかりません。")
            print("👆 上の「🎨 スキン生成」ボタンを先に押してください。")
    except Exception as e:
        print(f"❌ エラーが発生しました: {e}")

show_detailed_info()